In [1]:
import pygris
from pygris.data import get_census
import pandas as pd
from states import states

In [2]:
variables = {
    "S0801_C01_003E": "Drove Alone",
    "S0801_C01_046E": "Mean travel time",
    "S2001_C01_013E": "Median earnings",
}

In [3]:
data = []
for s in states:
    data.append(get_census(dataset = "acs/acs5/subject",
                               variables = list(variables.keys()),
                               year = 2022,
                               params = {
                                 "for": "tract:*",
                                 "in": f"state:{s}"
                               },
                               guess_dtypes = True,
                               return_geoid = True
                          )
    )
data = pd.concat(data)

In [4]:
data.rename(columns=variables, inplace=True)

In [5]:
tracts = pygris.tracts(cb = True, year=2022)

Retrieving Census tracts for the entire United States


In [6]:
tracts = tracts.merge(data, on="GEOID", how="inner")

In [7]:
# Mean Daily drive time, all trip purposes
tracts["Component 1"] = (tracts["Mean travel time"] * (tracts["Drove Alone"]/100) * 2 * 3 )/60

In [8]:
# Hours worked to afford a car spread over 7 days
tracts["Component 2"] = (12182/(tracts["Median earnings"]/40))/7

In [9]:
tracts["Total Car Ownership Time"] = tracts["Component 1"] + tracts["Component 2"]

In [10]:
tracts.to_file("tracts_time_wasted_on_cars.geojson")